In [43]:
#!/usr/bin/env python3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from autorank import autorank, create_report, plot_stats 
import json
import glob
import re
import os

# Configuration
algorithms = ["Adam", "MAML", "Approx_GAP", "Beta_cd", "MICAT"]
datasets = ["algebra", "assist0910", "math2"]
subalgos = ["IMPACT", "NCDM"]
metrics = ["mi_acc","rmse","mae","mi_prec","mi_rec","mi_f_b","mi_auc","ma_prec","ma_rec","ma_f_b",'meta_doa','pc-er','rm']
pattern = re.compile(
    r"^CAT_launch_"
    r"(?P<dataset>.+?)_"
    r"(?P<subalgo>.+?)_"
    r"(?P<algorithm>.+?)_"
    r"\d+_all_results\.json$"
)

output_dir = "cd_plots"
os.makedirs(output_dir, exist_ok=True)

In [73]:
# Collect metric values across all valid datasets/subalgos/folds/steps
results = {m: {a: [] for a in algorithms} for m in metrics}
files = glob.glob("CAT_launch_*_IMPACT_*_all_results.json") + \
        glob.glob("CAT_launch_*_NCDM_*_all_results.json")
print(f"[INFO] Found {len(files)} candidate result files.")

for file in files:
    m = pattern.match(os.path.basename(file))
    if not m:
        print(f"[WARN] Filename does not match expected pattern: {file}")
        continue
    ds, subalgo, algo = m["dataset"], m["subalgo"], m["algorithm"]
    if ds not in datasets or algo not in algorithms:
        print(f"[WARN] Skipping file due to unmatched dataset/algo: {ds}, {algo}")
        continue
    if ds=="algebra" and subalgo=="NCDM":
        continue
    try:
        with open(file, "r") as f:
            folds = json.load(f)
        for fold_idx, (pred, meta) in enumerate(folds):
            value_acc = {metric:0 for metric in metrics}
            for step in pred:
                for metric in metrics:
                    value = pred[step].get(metric)
                    if value is None or np.isnan(value):
                        value = meta[step].get(metric)
                    if value is not None and not np.isnan(value):
                        value_acc[metric] += value 
                    elif metric == 'pc-er' :
                        value_acc[metric] += -2 
            for metric in metrics :
                results[metric][algo].append(value_acc[metric])
    except Exception as e:
        print(f"[ERROR] Failed to process {file}: {e}")

[INFO] Found 28 candidate result files.


In [74]:
for algo, r in results['meta_doa'].items() :
    print(algo, len(r))
    """if len(r)<25:
        print(r)"""

Adam 25
MAML 25
Approx_GAP 25
Beta_cd 25
MICAT 25


In [75]:
results['pc-er']

{'Adam': [0.8447360992431641,
  0.8262834548950195,
  2.338153839111328,
  3.11041259765625,
  3.8050050735473633,
  2.78619384765625,
  3.126953125,
  3.295562744140625,
  2.89068603515625,
  4.04150390625,
  1.0103988647460938,
  1.351776123046875,
  1.640533447265625,
  0.5499649047851562,
  1.701690673828125,
  -1.46697998046875,
  0.018201828002929688,
  -0.53619384765625,
  -0.18083953857421875,
  -0.3705291748046875,
  0.6533432006835938,
  -0.352780818939209,
  0.000244140625,
  0.5296516418457031,
  0.7164802551269531],
 'MAML': [5.91015625,
  5.3782958984375,
  4.01300048828125,
  3.25701904296875,
  7.31591796875,
  -0.3679656982421875,
  2.6944427490234375,
  -4.344207763671875,
  4.271202087402344,
  0.985504150390625,
  9.220703125,
  3.8074951171875,
  10.06689453125,
  6.5843505859375,
  8.89990234375,
  -2.17578125,
  1.50006103515625,
  -1.453216552734375,
  -1.20819091796875,
  3.0425949096679688,
  -0.145263671875,
  -2.780029296875,
  2.8943023681640625,
  3.817413

In [69]:
from scipy.stats import friedmanchisquare
import numpy as np
import scikit_posthocs as sp

t = [results['mi_acc'][algo] for algo in algorithms]
data = np.array(t)

from critdd import Diagram
import pandas as pd

# download example data


# create a CD diagram from the Pandas DataFrame
diagram = Diagram(
    data.T,
    treatment_names = algorithms,
    maximize_outcome = True
)

# inspect average ranks and groups of statistically indistinguishable treatments
diagram.average_ranks # the average rank of each treatment
diagram.get_groups(alpha=.05, adjustment="holm")

# export the diagram to a file
diagram.to_file(
    "example.tex",
    alpha = .05,
    adjustment = "holm",
    reverse_x = True,
    axis_options = {"title": ""},
)

In [78]:
from critdd import Diagrams # Diagrams is the 2D version of Diagram
import numpy as np
import pandas as pd


# construct a sequence of CD diagrams
treatment_names = ["Naïve", "MAML", "Approx-GAP", "BETA-CD", "MICAT"]
metrics = ["mi_acc","mi_prec","mi_rec","mi_f_b","mi_auc",'meta_doa']
diagram_names = ["Accuracy","Precision","Recall","F1","AUC",'Meta Doa']
Xs = [] # collect an (n,k)-shaped matrix for each diagram
for n in metrics:
    Xs.append(np.array([results[n][algo] for algo in algorithms]).T)
two_dimensional_diagram = Diagrams(
    np.stack(Xs),
    diagram_names = diagram_names,
    treatment_names = treatment_names,
    maximize_outcome = True
)



In [79]:
two_dimensional_diagram.to_file(
    "2D_diagram.tex",
    alpha = .05,
    adjustment = "holm",
    reverse_x = True,
    axis_options = {"title": ""},
)

In [ ]:
#
# customize the style of the plot and export to PDF
two_dimensional_diagram.to_file(
    "2d_example.pdf",
    preamble = "\n".join([ # colors are defined before \begin{document}
        "\\definecolor{color1}{HTML}{84B818}",
        "\\definecolor{color2}{HTML}{D18B12}",
        "\\definecolor{color3}{HTML}{1BB5B5}",
        "\\definecolor{color4}{HTML}{F85A3E}",
        "\\definecolor{color5}{HTML}{4B6CFC}",
    ]),
    axis_options = { # style the plot
        "cycle list": ",".join([ # define the markers for treatments
            "{color1,mark=*}",
            "{color2,mark=diamond*}",
            "{color3,mark=triangle,semithick}",
            "{color4,mark=square,semithick}",
            "{color5,mark=pentagon,semithick}",
        ]),
        "width": "\\axisdefaultwidth",
        "height": "0.75*\\axisdefaultheight",
        "title": "critdd"
    },
)

In [28]:
# TIMETABLE

#!/usr/bin/env python3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from autorank import autorank, create_report, plot_stats 
import json
import glob
import re
import os

# Configuration
algorithms = ["Adam", "Approx_GAP", "Beta_cd", "MICAT", "MAML"]
datasets = ["algebra", "assist0910", "math2"]
subalgos = ["IMPACT", "NCDM"]
metrics = ["mi_acc","rmse","mae","mi_prec","mi_rec","mi_f_b","mi_auc","ma_prec","ma_rec","ma_f_b",'meta_doa','pc-er','rm']
pattern = re.compile(
    r"^CAT_launch_"
    r"(?P<dataset>.+?)_"
    r"(?P<subalgo>.+?)_"
    r"(?P<algorithm>.+?)_"
    r"\d+_metrics_summary\.csv$"
)

output_dir = "time_table"
os.makedirs(output_dir, exist_ok=True)

In [33]:
import csv
# Collect metric values across all valid datasets/subalgos/folds/steps
results = {a: {d: {"IMPACT":[0,0], "NCDM":[0,0]} for d in datasets} for a in algorithms}
files = glob.glob("CAT_launch_*_IMPACT_*_metrics_summary.csv") + \
        glob.glob("CAT_launch_*_NCDM_*_metrics_summary.csv")
print(f"[INFO] Found {len(files)} candidate result files.")

for file in files:
    m = pattern.match(os.path.basename(file))
    if not m:
        print(f"[WARN] Filename does not match expected pattern: {file}")
        continue
    ds, subalgo, algo = m["dataset"], m["subalgo"], m["algorithm"]
    if ds not in datasets or algo not in algorithms:
        print(f"[WARN] Skipping file due to unmatched dataset/algo: {ds}, {algo}")
        continue
    try:
        print(ds, subalgo, algo)
        with open(file, newline='') as f:
            for row in f:
                if row[0]=='-' : 
                    s = row.split(',')
                    results[algo][ds][subalgo][0] = float(s[2])/60
                    results[algo][ds][subalgo][1] = float(s[3])/60
                
                    
    except Exception as e:
        print(f"[ERROR] Failed to process {file}: {e}")

[INFO] Found 28 candidate result files.
math2 IMPACT Adam
math2 IMPACT Approx_GAP
math2 IMPACT Beta_cd
algebra IMPACT MAML
assist0910 IMPACT Adam
algebra IMPACT Adam
math2 IMPACT MAML
algebra IMPACT MICAT
assist0910 IMPACT Approx_GAP
assist0910 IMPACT Beta_cd
algebra IMPACT Approx_GAP
algebra IMPACT Beta_cd
math2 IMPACT MICAT
assist0910 IMPACT MICAT
assist0910 IMPACT MAML
math2 NCDM MAML
algebra NCDM Adam
assist0910 NCDM Adam
algebra NCDM Beta_cd
assist0910 NCDM Beta_cd
assist0910 NCDM MAML
math2 NCDM MICAT
math2 NCDM Approx_GAP
assist0910 NCDM Approx_GAP
math2 NCDM Adam
assist0910 NCDM MICAT
math2 NCDM Beta_cd
algebra NCDM Approx_GAP


In [34]:
results

{'Adam': {'algebra': {'IMPACT': [0.0, 0.0], 'NCDM': [0.0, 0.0]},
  'assist0910': {'IMPACT': [0.0, 0.0], 'NCDM': [0.0, 0.0]},
  'math2': {'IMPACT': [0.0, 0.0], 'NCDM': [0.0, 0.0]}},
 'Approx_GAP': {'algebra': {'IMPACT': [15.636566666666669, 1.9345521969638804],
   'NCDM': [4.5280000000000005, 0.7119747186522846]},
  'assist0910': {'IMPACT': [6.306100000000001, 0.39740983603551855],
   'NCDM': [10.202599999999999, 1.0624886182396915]},
  'math2': {'IMPACT': [65.5274, 31.93663734964559],
   'NCDM': [61.630766666666666, 7.737920111194109]}},
 'Beta_cd': {'algebra': {'IMPACT': [12.9396, 2.777689848997224],
   'NCDM': [60.49596666666667, 2.314893555268193]},
  'assist0910': {'IMPACT': [15.885433333333333, 2.6775153454275484],
   'NCDM': [29.354866666666666, 8.701718447301722]},
  'math2': {'IMPACT': [84.26469999999999, 5.656020853430832],
   'NCDM': [163.21325, 56.15742929553786]}},
 'MICAT': {'algebra': {'IMPACT': [14.3494, 3.8294388887639625],
   'NCDM': [0, 0]},
  'assist0910': {'IMPACT':

In [ ]:
{'Naive': {'algebra': {'IMPACT': [0, 0], 'NCDM': [0, 0]},
  'assist0910': {'IMPACT': [0.0, 0.0], 'NCDM': [0.0, 0.0]},
  'math2': {'IMPACT': [0.0, 0.0], 'NCDM': [0.0, 0.0]}},
 'Approx_GAP': {'algebra': {'IMPACT': [0, 0], 'NCDM': [0, 0]},
  'assist0910': {'IMPACT': [6.306100000000001, 0.39740983603551855],
   'NCDM': [10.202599999999999, 1.0624886182396915]},
  'math2': {'IMPACT': [65.5274, 31.93663734964559],
   'NCDM': [61.630766666666666, 7.737920111194109]}},
 'Beta_cd': {'algebra': {'IMPACT': [0, 0], 'NCDM': [0, 0]},
  'assist0910': {'IMPACT': [15.885433333333333, 2.6775153454275484],
   'NCDM': [29.354866666666666, 8.701718447301722]},
  'math2': {'IMPACT': [84.26469999999999, 5.656020853430832],
   'NCDM': [163.21325, 56.15742929553786]}},
 'MICAT': {'algebra': {'IMPACT': [0, 0], 'NCDM': [0, 0]},
  'assist0910': {'IMPACT': [11.7725, 1.987660950855441],
   'NCDM': [20.399400000000004, 0.3442023710293439]},
  'math2': {'IMPACT': [54.6772, 6.302373202038877], 'NCDM': [0, 0]}},
 'MAML': {'algebra': {'IMPACT': [0, 0], 'NCDM': [0, 0]},
  'assist0910': {'IMPACT': [8.475566666666666, 2.013736802839713],
   'NCDM': [24.715366666666668, 10.257397938263335]},
  'math2': {'IMPACT': [106.3803, 24.24575084748528],
   'NCDM': [51.81773333333334, 2.838770262905325]}}}